In [1]:
import pandas as pd
import re 

In [2]:

bank_statement_df = pd.read_excel(r"D:\NPS_phase1\nps-reconciliation\output\BANK STMT PHASE 4\Bank_bestfinance.xlsx", skiprows=1)


In [3]:
bank_statement_df.head()

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,915103010100,-86916.28,DR,1,PRABHUMONEY#PMT91752,NPS-IF-3196423,2023-09-05T00:00:00,NPS131312


In [9]:
for index, row in bank_statement_df.iterrows():
    if "NPS-IF-" in str(row["Desc3"]):
        id = re.findall(r'[0-9]{7}',str(row["Desc3"]))
        bank_statement_df.loc[index, 'Transaction Id'] = id[0]

    elif "10000" in str(row["Desc3"]):
        id = re.findall(r'10*[0-9]{7}',str(row["Desc3"]))
        bank_statement_df.loc[index, 'Transaction Id'] = id[0].replace("10000","")
    
    elif "FTMS-" in str(row["Desc3"]):
        match = re.search(r'FTMS-(\d+)',str(row["Desc3"]))
        if match:
            id= match.group(1)
            bank_statement_df.loc[index, 'Transaction Id'] = id

In [8]:
bank_statement_df['Transaction Id']

0    3196423
Name: Transaction Id, dtype: object

In [10]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%Y-%m-%dT%H:%M:%S").dt.date

In [11]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction ID,Transaction Id
0,1,915103010100,-86916.28,DR,1,PRABHUMONEY#PMT91752,NPS-IF-3196423,2023-09-05,NPS131312,3196423,3196423


In [12]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'DR' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'CR':
            mode = 'CR'
            amount = abs(row['Amount'])
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [13]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)


In [14]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction ID,Transaction Id,Mode
0,1,915103010100,86916.28,DR,1,PRABHUMONEY#PMT91752,NPS-IF-3196423,2023-09-05,NPS131312,3196423,3196423,DR
